### Imports

In [22]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

### Data

In [17]:
data = np.array ([0])

### Loss function

In [18]:
def L(data):
    return data

### Kernel

In [61]:
def k(x, y, N):
    for n in range(N):
        qml.RX(x, wires=n)
        qml.adjoint(qml.RX(y, wires=n))
    return qml.expval(qml.PauliZ(wires=range(N)))
    #return qml.expval(qml.s_prod(1/2,qml.sum(qml.Identity(wires=range(N)),qml.PauliZ(wires=range(N)))))

In [62]:
circuit = qml.QNode(k, qml.device('default.qubit', wires=2))


In [63]:
print(qml.draw(circuit) (np.pi/4, np.pi/4, 2))

ValueError: PauliZ: wrong number of wires. 2 wires given, 1 expected.